In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 5s (53.4 kB/s)
Reading package li

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-06-19 07:52:56--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.57MB/s    in 0.2s    

2022-06-19 07:52:57 (5.57 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
user_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Music_v1_00.tsv.gz"), sep="\t", header=True)

# Show DataFrame
user_data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|          Five Stars|Love this CD alon...| 2015-08-31|
|         US|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|     831769051|Same Trailer Diff...|           Music|          5|    

In [5]:
dg = user_data_df.select("review_id", "customer_id", "product_id", "product_parent", "review_date")
dg.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3LI5TRP3YIDQL|   10140119|B00TXH4OLC|     384427924| 2015-08-31|
|R3LGC3EKEG84PX|   27664622|B00B6QXN6U|     831769051| 2015-08-31|
| R9PYL3OYH55QY|   45946560|B001GCZXW6|      14067376| 2015-08-31|
|R3PWBAWUS4NT0Q|   15146326|B000003EK6|     566295619| 2015-08-31|
|R15LYP3O51UU9E|   16794688|B00N1F0BKK|     210426072| 2015-08-31|
|R1AD7L0CC3DSRI|   32203364|B00V7KAO7Q|     360249504| 2015-08-31|
|R32FE8Y45QV434|    1194276|B000094Q4P|      20972283| 2015-08-31|
|R3NM4MZ4XWL43Q|   45813052|B00JMK0P1I|     378049560| 2015-08-31|
|R3H4FXX6Q7I37D|   12795687|B008OW1S3O|     141620702| 2015-08-31|
|R30L5PET7LFFDC|   36673840|B00VI2L3L4|     920557135| 2015-08-31|
|  REFRE1LEKLAF|   49453576|B0000041EV|     683128964| 2015-08-31|
|R3JTJ5EQN74E9H|    3285047|B00005YW4H|     268461622| 2015-08

In [6]:
user = user_data_df.select("product_id", "product_title")
user.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00TXH4OLC|Whatever's for Us...|
|B00B6QXN6U|Same Trailer Diff...|
|B001GCZXW6| Soaring (Jazz Club)|
|B000003EK6|     CARIBBEAN PARTY|
|B00N1F0BKK|         Pain Killer|
|B00V7KAO7Q|A Thoughtiverse U...|
|B000094Q4P|Comfort of Her Wings|
|B00JMK0P1I|Sleep Soundly CD ...|
|B008OW1S3O|     An Awesome Wave|
|B00VI2L3L4|          Blurryface|
|B0000041EV|Mozart For Mother...|
|B00005YW4H|   Come Away with Me|
|B00Q9KEZV0| Tomorrow Is My Turn|
|B00GFXRKHW|       The Outsiders|
|B004L3AQ10|  The Lincoln Lawyer|
|B0000025PM|Bach: Goldberg Va...|
|B00701QUN8|Quiet Riot - Grea...|
|B003UJC2QI|Love Or Money/Lov...|
|B00LGA16J2|Jonas Kaufmann - ...|
|B00ZGVMLLE|        Clockworking|
+----------+--------------------+
only showing top 20 rows



In [7]:
user.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [8]:
user_count = user_data_df.groupBy("customer_id").count()
user_count.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   16794688|    3|
|   49997672|    5|
|   16066470|    1|
|     636526|    1|
|   15963400|    8|
|   47840769|    4|
|    5884899|    7|
|   31220419|    2|
|   25410451|    8|
|   51052113|    4|
|   31562327|    2|
|   40710396|    5|
|   50988780|    4|
|   12374662|    4|
|   37007490|    1|
|    7328230|    1|
|   51154721|    1|
|   37153007|    1|
|   12318800|    2|
|   33775903|    1|
+-----------+-----+
only showing top 20 rows



In [9]:
vine = user_data_df.select("review_id", "star_rating", "helpful_votes", "total_votes", "vine")
vine.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3LI5TRP3YIDQL|          5|            0|          0|   N|
|R3LGC3EKEG84PX|          5|            0|          0|   N|
| R9PYL3OYH55QY|          5|            0|          1|   N|
|R3PWBAWUS4NT0Q|          3|            0|          0|   N|
|R15LYP3O51UU9E|          5|            0|          0|   N|
|R1AD7L0CC3DSRI|          5|            0|          0|   N|
|R32FE8Y45QV434|          5|            0|          0|   N|
|R3NM4MZ4XWL43Q|          5|            1|          2|   N|
|R3H4FXX6Q7I37D|          4|            0|          0|   N|
|R30L5PET7LFFDC|          5|            1|          1|   N|
|  REFRE1LEKLAF|          4|            1|          1|   N|
|R3JTJ5EQN74E9H|          5|            0|          0|   N|
|R1W2F091LCOAW5|          5|            0|          0|   N|
| RYUMFQRRB1FNM|          3|            

In [10]:
user_data_df.count()

4751577

In [11]:
mode = "append"
jdbc_url = "jdbc:postgresql://mypostgredb.c7mjgogjvx7f.us-east-1.rds.amazonaws.com:5432/postgres"
config = {
    "user":"postgres",
    "password":"Crayon-360",
    "driver":"org.postgresql.Driver"
}

In [12]:
dg.write.jdbc(url=jdbc_url, table="usernami", mode=mode, properties=config)

KeyboardInterrupt: ignored

In [ ]:
user.write.jdbc(url=jdbc_url, table="user_active", mode=mode, properties=config)

In [ ]:
user_count.write.jdbc(url=jdbc_url, table="user_count", mode=mode, properties=config)

In [ ]:
vine.write.jdbc(url=jdbc_url, table="vine", mode=mode, properties=config)